# 72 - Edit Distance

Given two strings word1 and word2, return the minimum number of operations required to convert word1 to word2.

You have the following three operations permitted on a word:

- Insert a character
- Delete a character
- Replace a character
 

## Example 1:

- Input: word1 = "horse", word2 = "ros"
- Output: 3
- Explanation: 
```
horse -> rorse (replace 'h' with 'r')
rorse -> rose (remove 'r')
rose -> ros (remove 'e')
```

## Example 2:

- Input: word1 = "intention", word2 = "execution"
- Output: 5
- Explanation: 
```
intention -> inention (remove 't')
inention -> enention (replace 'i' with 'e')
enention -> exention (replace 'n' with 'x')
exention -> exection (replace 'n' with 'c')
exection -> execution (insert 'u')
```

# Base Version

In [1]:
def minDistance(word1: str, word2: str) -> int:
    n, m = len(word1), len(word2)
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    
    # Base cases
    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j

    # Fill DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(
                    dp[i - 1][j],    # delete
                    dp[i][j - 1],    # insert
                    dp[i - 1][j - 1] # replace
                )
    return dp[n][m]
minDistance("horse", "ros")  # 3

3

# Verbose Version

In [3]:
def edit_distance_verbose(word1: str, word2: str) -> int:
    n, m = len(word1), len(word2)
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    # Base cases
    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j

    def print_table(dp, i=None, j=None):
        """Pretty-print the DP table, highlighting dp[i][j] if given."""
        header = ["    "] + [" ''"] + [f" {c}" for c in word2]
        print("".join(header))
        for r in range(n + 1):
            row_label = " ''" if r == 0 else f" {word1[r-1]}"
            line = [f"{row_label:>4}"]
            for c in range(m + 1):
                val = dp[r][c]
                cell = f"[{val}]" if (i == r and j == c) else f" {val} "
                line.append(f"{cell:>4}")
            print("".join(line))
        print()

    print("Initial DP table with base cases:")
    print_table(dp)

    # Fill DP
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            a, b = word1[i - 1], word2[j - 1]
            if a == b:
                dp[i][j] = dp[i - 1][j - 1]
                print(f"i={i}, j={j} | '{a}' == '{b}' → no cost, copy dp[{i-1}][{j-1}] = {dp[i][j]}")
            else:
                delete = dp[i - 1][j]
                insert = dp[i][j - 1]
                replace = dp[i - 1][j - 1]
                dp[i][j] = 1 + min(delete, insert, replace)
                choice = "delete" if delete <= insert and delete <= replace else (
                    "insert" if insert <= delete and insert <= replace else "replace"
                )
                print(f"i={i}, j={j} | '{a}' != '{b}' → choose {choice}")
                print(f"dp[{i}][{j}] = 1 + min({delete}, {insert}, {replace}) = {dp[i][j]}")
            print_table(dp, i, j)

    print(f"Final Edit Distance = {dp[n][m]}")
    return dp[n][m]

# ---------- Small Demo ----------
if __name__ == "__main__":
    w1, w2 = "horse", "rose"
    print(f"word1 = '{w1}', word2 = '{w2}'\n")
    ans = edit_distance_verbose(w1, w2)
    print(f"\nAnswer: {ans}")


word1 = 'horse', word2 = 'rose'

Initial DP table with base cases:
     '' r o s e
  ''  0   1   2   3   4 
   h  1   0   0   0   0 
   o  2   0   0   0   0 
   r  3   0   0   0   0 
   s  4   0   0   0   0 
   e  5   0   0   0   0 

i=1, j=1 | 'h' != 'r' → choose replace
dp[1][1] = 1 + min(1, 1, 0) = 1
     '' r o s e
  ''  0   1   2   3   4 
   h  1  [1]  0   0   0 
   o  2   0   0   0   0 
   r  3   0   0   0   0 
   s  4   0   0   0   0 
   e  5   0   0   0   0 

i=1, j=2 | 'h' != 'o' → choose insert
dp[1][2] = 1 + min(2, 1, 1) = 2
     '' r o s e
  ''  0   1   2   3   4 
   h  1   1  [2]  0   0 
   o  2   0   0   0   0 
   r  3   0   0   0   0 
   s  4   0   0   0   0 
   e  5   0   0   0   0 

i=1, j=3 | 'h' != 's' → choose insert
dp[1][3] = 1 + min(3, 2, 2) = 3
     '' r o s e
  ''  0   1   2   3   4 
   h  1   1   2  [3]  0 
   o  2   0   0   0   0 
   r  3   0   0   0   0 
   s  4   0   0   0   0 
   e  5   0   0   0   0 

i=1, j=4 | 'h' != 'e' → choose insert
dp[1][4] = 1 + m

## Why Each Case Represents Delete / Insert / Replace

### **dp[i-1][j]** → DELETE

Think of it like this:

- You are at dp[i][j], meaning you have word1[:i] and word2[:j]. 

- If you delete word1[i-1], you shrink word1 to length i-1.

- Now you just need to convert word1[:i-1] → word2[:j], which is exactly dp[i-1][j].

- Since you performed a delete, add 1.

### **dp[i][j-1]** → INSERT

This time:

- You want to match word2[j-1].

- If you insert word2[j-1] into word1, your new string’s prefix matches up to j-1.

- Now you just need to convert word1[:i] → word2[:j-1], which is dp[i][j-1].

- Add 1 because you just inserted a character.

### **dp[i-1][j-1]** → REPLACE

Here:

- You replace word1[i-1] with word2[j-1].

- After this replacement, you have matched both last characters.

- Now you just need to convert word1[:i-1] → word2[:j-1] (which is dp[i-1][j-1]).

- Again, add 1 because you did a replacement.


## Dive Deeper:

### 1. DELETE = dp[i-1][j]

If we choose delete, we are saying:

- "I will remove the last char of word1[:i] (i.e. word1[i-1]).
Then I just need to convert the shorter string word1[:i-1] → word2[:j]."

- Visually, we move up in the DP table because we reduce i by 1 (word1 got shorter) but j stays the same (word2 unchanged):

```
dp[i][j] = 1 + dp[i-1][j]
```

Example:
```
At (i=2,j=1) ("ho" → "r"),
we try deleting 'o', leaving "h" → "r".
Cost = 1 (delete) + dp[1][1] (cost of converting "h" → "r").
```
That’s why we look up — we've shrunk word1 by 1.

### 2. INSERT = dp[i][j-1]

If we choose insert, we are saying:

- "I will insert word2[j-1] at the end of word1[:i]
so that they match on this new char.
Now I just need to convert word1[:i] → word2[:j-1] (the rest)."

- Visually, we move left in the DP table because we reduce j by 1 (word2 got shorter) but i stays the same (word1 unchanged):

```
dp[i][j] = 1 + dp[i][j-1]
```

Example:
```
At (i=1,j=2) ("h" → "ro"),
we try inserting 'o' after "h" (becomes "ho").
Cost = 1 (insert) + dp[1][1] (cost of converting "h" → "r").
```
We look left because we "consume" the last char of word2 by inserting it, so we now just match the earlier part.

### 3. REPLACE = dp[i-1][j-1]

If we choose replace, we are saying:

- "I will replace word1[i-1] with word2[j-1]
and then I just need to convert the first i-1 chars of word1 → first j-1 chars of word2."

- Visually, we move diagonally up-left in the table because we shrink both prefixes by 1 (we consumed both last characters):
```
dp[i][j] = 1 + dp[i-1][j-1]
```

Example:
```
At (i=1,j=1) ("h" → "r"),
replace 'h' with 'r'.
Cost = 1 (replace) + dp[0][0] (cost of converting empty → empty = 0).
```